In [1]:
from research.model_pick.candle_fetch import FusionCandles
from research.model_pick.labeler import PipelineLabeler
from src.utils.drop_na import drop_na_and_align_x_and_y

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles("2022-07-01", "2025-10-25")

labeler = PipelineLabeler(candles, 4)
label_for_regressor = labeler.label_direction

PyTorch configured: device=cpu, dtype=torch.float32


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from research.model_pick.features import FeatureLoader

feature_loader = FeatureLoader(candles)
df_feat, label_r = feature_loader.get_feature_label_bundle(
    label_for_regressor, 1
)

In [3]:
from jesse.helpers import date_to_timestamp

train_mask = df_feat.index.to_numpy() < date_to_timestamp("2025-04-30")
train_x_all_feat = df_feat[train_mask]
train_y = label_r[train_mask]

In [4]:
from pathlib import Path
from research.model_pick.feature_select import FeatureSelector

MODEL_SAVE_DIR = Path("strategies/BinanceBtcDemoBarV2/models")
feature_selector = FeatureSelector(model_save_dir=MODEL_SAVE_DIR)
df_feat = feature_selector.get_all_features(train_x_all_feat)

df_feat.shape

Epoch 10/100 | Train Loss: 108.4064
Epoch 20/100 | Train Loss: 100.1268
Epoch 30/100 | Train Loss: 94.1508
Epoch 40/100 | Train Loss: 87.1805
Epoch 50/100 | Train Loss: 75.7657
Epoch 60/100 | Train Loss: 64.1939
Epoch 70/100 | Train Loss: 57.3556
Epoch 80/100 | Train Loss: 53.5787
Epoch 90/100 | Train Loss: 51.3759
Epoch 100/100 | Train Loss: 49.8682
Model saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDemoBarV2/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDemoBarV2/models/deep_ssm.json
Epoch 10/100 | ELBO: -252.3625
Epoch 20/100 | ELBO: -179.6739
Epoch 30/100 | ELBO: -138.6002
Epoch 40/100 | ELBO: -118.1968
Epoch 50/100 | ELBO: -102.1949
Epoch 60/100 | ELBO: -83.5398
Epoch 70/100 | ELBO: -69.7226
Epoch 80/100 | ELBO: -61.6460
Epoch 90/100 | ELBO: -58.6284
Epoch 100/100 | ELBO: -55.8108
Model saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDemoBarV2/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jes

(24196, 33591)

In [5]:
train_y.shape

(24196,)

In [7]:
from src.utils.drop_na import drop_na_and_align_x_and_y

x, y = drop_na_and_align_x_and_y(df_feat, train_y)

In [10]:
x.head()

,deep_ssm_0,deep_ssm_1,deep_ssm_2,deep_ssm_3,deep_ssm_4,lg_ssm_0,lg_ssm_1,lg_ssm_2,lg_ssm_3,lg_ssm_4,...,vmd_w256_1_ddt_lag1,vmd_w256_1_ddt_lag2,vmd_w256_1_ddt_lag3,vmd_w256_1_ddt_lag4,vmd_w256_1_ddt_lag5,vmd_w256_2_ddt_lag1,vmd_w256_2_ddt_lag2,vmd_w256_2_ddt_lag3,vmd_w256_2_ddt_lag4,vmd_w256_2_ddt_lag5
1711370640000,-0.651157,-1.642009,0.615258,-2.693879,2.149300,0.595300,0.325413,-0.561480,-0.374559,-1.434780,...,48.610814,89.552815,-128.539568,96.994923,-76.007654,-98.131012,139.518269,-140.628693,122.946438,-102.710919
1711373460000,-0.395071,-0.760368,-1.010063,-3.783730,1.353403,0.733095,0.196541,-0.314192,1.351922,0.083569,...,-73.860121,48.610814,89.552815,-128.539568,96.994923,27.168639,-98.131012,139.518269,-140.628693,122.946438
1711374660000,-2.657255,-0.503824,0.929034,-2.464360,1.102957,-1.164451,-1.222124,-0.635316,0.675966,1.264949,...,-56.276643,-73.860121,48.610814,89.552815,-128.539568,40.666088,27.168639,-98.131012,139.518269,-140.628693
1711375080000,-1.526237,1.230861,1.361303,-1.608522,2.728820,-1.453627,-2.006150,0.138157,0.446415,-0.871462,...,91.170959,-56.276643,-73.860121,48.610814,89.552815,-74.043465,40.666088,27.168639,-98.131012,139.518269
1711375200000,-0.300113,0.185436,2.575488,-2.739291,0.366903,-1.907860,-0.603853,-0.127619,1.867789,-1.205365,...,58.853283,91.170959,-56.276643,-73.860121,48.610814,83.126756,-74.043465,40.666088,27.168639,-98.131012


In [9]:
x.isna().sum().sort_values(ascending=False)

fti_1_skew20             3673
fti_1_kurt20             3673
fti_1_skew20_lag1        3672
fti_1_kurt20_lag1        3672
fti_1_skew20_lag2        3671
                         ... 
cwt_w64_2_mean20_lag5       0
cwt_w64_2_mean20_lag4       0
cwt_w64_2_mean20_lag3       0
cwt_w64_2_mean20_lag2       0
vmd_w256_2_ddt_lag5         0
Length: 33591, dtype: int64